### Experimentation with FLAN T5 (most likely will not use)

In [1]:
# !pip install transformers
# !pip install tensorflow
# !pip install torch
# !pip install sentencepiece
# !pip install tf-keras

In [1]:
import requests
import xml.etree.ElementTree as ET
from bs4 import BeautifulSoup
from transformers import T5Tokenizer, T5ForConditionalGeneration, pipeline
import pandas as pd
from tqdm import tqdm

# Distil BERT Sentiment Analysis

In [19]:
model_name='distilbert/distilbert-base-uncased-finetuned-sst-2-english'
revision='af0f99b'
classifier = pipeline(task="sentiment-analysis", model=model_name, revision=revision)

C:\Users\blkeu\anaconda3\envs\CS172B311\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [3]:
df=pd.read_csv("data/4.12.15-5.11.24_articles.csv")

In [4]:
data = []
for index, row in df.iterrows():
    date_only = row['Date and Time'].split('T')[0]
    data.append({'title': row['Article Title'], 'publishedAt': date_only})

df = pd.DataFrame(data, columns=['title', 'publishedAt'])

In [5]:
df['publishedAt'] = df['publishedAt'].astype(str)

df['title'] = df['title'].apply(lambda x: str(x))

# Group by 'publishedAt' and aggregate 'title' into lists
grouped_df = df.groupby('publishedAt', as_index=False).agg({'title': list})

# Rename columns for clarity
grouped_df.columns = ['date', 'article_titles']

In [6]:
grouped_df

,date,article_titles
0,2015-04-11,[Analyst: Market Is Wrong On Threat To Garmin ...
1,2015-04-12,[Why Apple Watch Is Just 'Noise' And Apple Inv...
2,2015-04-14,"[Will The $17,000 Apple Watch Inspire A $20,00..."
3,2015-04-15,"[6% Of US Adults Plan To Purchase Apple Watch,..."
4,2015-04-17,[Apple Offers Workshops For Users To Get Acqua...
...,...,...
2010,2024-05-05,"[Apple's Q2 Earnings, iPhone Sales Slump, And ..."
2011,2024-05-06,"[Thanks To Apple, This Analyst Is Optimistic O..."
2012,2024-05-08,[Apple Reverses Sales Decline in China with Im...
2013,2024-05-09,"[""Apple Apologizes For iPad 'Crush' Ad That 'M..."


In [9]:
output_data = []
for index, row in tqdm(grouped_df.iterrows()):
    preds = classifier(row['article_titles'])
    preds = [{"score": round(pred["score"], 4), "label": pred["label"]} for pred in preds]
    weighted_scores = []
    for pred in preds:
        score = pred['score']
        if pred['label'] == 'NEGATIVE':
            score = -score
        weighted_scores.append(score)
    
    overall_sentiment = sum(weighted_scores) / len(weighted_scores)
    overall_sentiment = round(overall_sentiment, 4)

    output_data.append([row['date'], row['article_titles'], overall_sentiment])

output_df = pd.DataFrame(output_data, columns=['date', 'article_titles', 'sentiment'])

2015it [06:23,  5.25it/s]


In [13]:
# for i in output_df['sentiment'].unique():
#     print(i)

In [14]:
output_df

,date,article_titles,sentiment
0,2015-04-11,[Analyst: Market Is Wrong On Threat To Garmin ...,-0.9976
1,2015-04-12,[Why Apple Watch Is Just 'Noise' And Apple Inv...,-0.9995
2,2015-04-14,"[Will The $17,000 Apple Watch Inspire A $20,00...",-0.5197
3,2015-04-15,"[6% Of US Adults Plan To Purchase Apple Watch,...",-0.7127
4,2015-04-17,[Apple Offers Workshops For Users To Get Acqua...,-0.2263
...,...,...,...
2010,2024-05-05,"[Apple's Q2 Earnings, iPhone Sales Slump, And ...",-0.9991
2011,2024-05-06,"[Thanks To Apple, This Analyst Is Optimistic O...",-0.0173
2012,2024-05-08,[Apple Reverses Sales Decline in China with Im...,-0.5756
2013,2024-05-09,"[""Apple Apologizes For iPad 'Crush' Ad That 'M...",-0.8515


In [15]:
output_df.to_csv('data/4.12.15-5.11.24_BERT_sentiments.csv', encoding='utf-8', index=False)